In [14]:
import pandas as pd
import numpy as np
import joblib

In [15]:
df = pd.read_csv('data/cs-training_ver02.csv')
x = df.drop(columns='seriousdlqin2yrs')
y = df.seriousdlqin2yrs

In [16]:
with open('baseline.pkl','rb') as f:
    data = joblib.load(f)
print(data.keys())
locals().update(data)

dict_keys(['grid_result', 'best_param', 'cv_result', 'best_baseline'])


In [17]:
best_param

{'전처리': MinMaxScaler(),
 '알고리즘__solver': 'lbfgs',
 '알고리즘__max_iter': 300,
 '알고리즘__C': 8.0,
 '알고리즘': LogisticRegression(C=8.0, max_iter=300)}

In [18]:
y.value_counts()

0.0    139974
1.0     10026
Name: seriousdlqin2yrs, dtype: int64

In [19]:
from sklearn.base import clone
new_model = clone(best_baseline, safe=False)

In [20]:
cv = 10
for i in range(10):
    x_target = x.loc[y[y==1].index]
    x_nontarget = x.loc[y[y!=1].index].sample(len(x_target))
    x_sample = pd.concat([x_target,x_nontarget])
    y_new = np.concatenate([np.array([1]*len(x_target)), np.array([0]*len(x_target))])
    new_model.fit(x_sample, y_new)

In [21]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(new_model, x, y, cv=4, scoring='roc_auc').mean())
print(cross_val_score(best_baseline, x, y, cv=4, scoring='roc_auc').mean())

0.8487194166095416
0.8487194166095416


In [10]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_over, y_over = smote.fit_sample(x,y)

In [12]:
new_model02 = clone(best_baseline)
new_model02.fit(x_over, y_over)

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘', LogisticRegression(C=8.0, max_iter=300))])

In [13]:
print(cross_val_score(new_model02, x, y, cv=4, scoring='roc_auc').mean())
print(cross_val_score(best_baseline, x, y, cv=4, scoring='roc_auc').mean())

0.8487194166095416
0.8487194166095416
